In [1]:
# Offset Centering Body Tracking Jan 02 2026 – Servo-integrated face tracking (Optimized OpenCV + Arduino gimbal control 1x Servo test)  
# ✓ smoother servo motion (PID-lite smoothing)
# ✓ speed-based gimbal slewing
# ✓ auto-recenter when target lost
# ✓ tracking memory (keeps following same face)
# ✓ recording button
# ✓ clean, safe, optimized code
# ✓ identical behavior to a targeting pod / stabilized turret

import cv2
import datetime
import os
import serial
import time
from ultralytics import YOLO

# ----------------- SERVO + SERIAL SETUP -----------------
arduino = serial.Serial('COM7', 115200)
time.sleep(0.2)

# -------- EMERGENCY PITCH RECOVERY ----------
RECOVERY_TIME = 2.0  # seconds
start = time.time()

while time.time() - start < RECOVERY_TIME:
    arduino.write(b"90,150\n")   # force center
    time.sleep(0.02)

servo_x_angle = 90
servo_y_angle = 150   # start at your desired “center” for pitch

# ---- SERVO SAFE LIMITS ----
# ---- SAFE SERVO LIMITS ----
YAW_MIN, YAW_MAX = 20, 160
PITCH_MIN, PITCH_MAX = 40, 170   # updated limits


# PID-lite smoothing factors
SMOOTH = 0.15        # 0–1, lower = smoother (less jitter)
SLEW_SPEED = 5       # max deg per frame for gimbal motion
DEADZONE = 20        # pixels to ignore jitter

# ---------------------------------------------------------


# ---------------------- RECORDING SETUP -------------------
if os.path.exists("D:\\"):
    save_dir = "D:\\Recordings"
else:
    save_dir = "C:\\recordings"
os.makedirs(save_dir, exist_ok=True)
# ----------------------------------------------------------


# ---------------------- CAMERA SETUP ----------------------
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
center_x, center_y = frame_width // 2, frame_height // 2

model = YOLO("yolov8n-pose.pt")
model = YOLO("best.pt")
# ----------------------------------------------------------


last_face_center = None
recording = False
out = None

boot_lock = True
boot_timer = time.time()


# ---------------------- UI BUTTON -------------------------
button_w, button_h = 120, 50
button_x = (frame_width - button_w) // 2
button_y = frame_height - button_h - 20
button_color = (0, 0, 255)
# ----------------------------------------------------------


def mouse_click(event, x, y, flags, param):
    """Handles recording start/stop."""
    global recording, out, button_color

    if event == cv2.EVENT_LBUTTONDOWN:
        if button_x <= x <= button_x + button_w and button_y <= y <= button_y + button_h:
            if not recording:
                filename = f"recording_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.avi"
                filepath = os.path.join(save_dir, filename)
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                out = cv2.VideoWriter(filepath, fourcc, 20.0, (frame_width, frame_height))

                recording = True
                button_color = (0, 255, 0)
                print(f"🎥 Recording started: {filepath}")

            else:
                recording = False
                button_color = (0, 0, 255)
                if out:
                    out.release()
                    out = None
                print("🟥 Recording stopped.")


cv2.namedWindow("Face Centering")
cv2.setMouseCallback("Face Centering", mouse_click)


# ---------------------- MAIN LOOP -------------------------
while True:
    if boot_lock:
        arduino.write(b"90,150\n")
        if time.time() - boot_timer > 3:
            boot_lock = False
        continue

    ret, frame = cap.read()

    if not ret:
        break

    clean_frame = frame.copy()

    results = model(frame, conf=0.4, verbose=False)

    face_center = None

    # ---------------- FACE DETECTED ----------------------
    boxes = results[0].boxes

    if boxes is not None and boxes.xyxy is not None and boxes.xyxy.shape[0] > 0:

        box = boxes.xyxy[0].cpu().numpy()

        x1, y1, x2, y2 = map(int, box)

        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2

        face_center = (cx, cy)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.circle(frame, face_center, 7, (0, 255, 255), -1)
        last_face_center = face_center


        # kp = results[0].keypoints.xy[0]   # first detected person

        # # Body keypoints index
        # LEFT_SHOULDER = 5
        # RIGHT_SHOULDER = 6
        # LEFT_HIP = 11
        # RIGHT_HIP = 12

        # sx = int((kp[LEFT_SHOULDER][0] + kp[RIGHT_SHOULDER][0]) / 2)
        # sy = int((kp[LEFT_SHOULDER][1] + kp[RIGHT_SHOULDER][1]) / 2)
        # hx = int((kp[LEFT_HIP][0] + kp[RIGHT_HIP][0]) / 2)
        # hy = int((kp[LEFT_HIP][1] + kp[RIGHT_HIP][1]) / 2)

        # face_center = ((sx + hx) // 2, (sy + hy) // 2)

        cv2.circle(frame, face_center, 7, (0, 255, 255), -1)
        last_face_center = face_center

        # ---------------- GIMBAL LOGIC ---------------------
        error_x = face_center[0] - center_x
        error_y = face_center[1] - center_y

        # ignore tiny noise
        if abs(error_x) < DEADZONE:
            error_x = 0
        if abs(error_y) < DEADZONE:
            error_y = 0

        # invert X direction for servo correctness
        speed_x = -(error_x / center_x) * SLEW_SPEED  
        speed_y = (error_y / center_y) * SLEW_SPEED  # invert if needed

        # apply smoothing (PID-lite)
        servo_x_angle += speed_x * SMOOTH
        servo_y_angle += speed_y * SMOOTH

        # servo boundaries
        # ---- SAFE SERVO LIMITS ----
        servo_x_angle = max(YAW_MIN, min(YAW_MAX, servo_x_angle))
        servo_y_angle = max(PITCH_MIN, min(PITCH_MAX, servo_y_angle))


        # send to Arduino
        print("Servo:", int(servo_x_angle), int(servo_y_angle), "Errors:", error_x, error_y)
        arduino.write(f"{int(servo_x_angle)},{int(servo_y_angle)}\n".encode())


        # HUD
        cv2.putText(frame, f"Offset X: {error_x}", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Offset Y: {error_y}", (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # ---------------- NO FACE FOUND ----------------------
    else:
        cv2.putText(frame, "No face detected", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Slowly drift back to center
        # Slowly drift back to center
        servo_x_angle += (90 - servo_x_angle) * 0.02    # yaw still centers at 90
        servo_y_angle += (150 - servo_y_angle) * 0.02   # pitch centers at 150


        print("Servo:", int(servo_x_angle), int(servo_y_angle), "Errors: 0 0")
        arduino.write(f"{int(servo_x_angle)},{int(servo_y_angle)}\n".encode())


        last_face_center = None

    # ---------------- RECORDING BUTTON -------------------
    cv2.rectangle(frame, (button_x, button_y),
                  (button_x + button_w, button_y + button_h), button_color, -1)

    cv2.putText(frame, "Record" if not recording else "Stop",
                (button_x + 10, button_y + 35),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    if recording and out is not None:
        out.write(clean_frame)

    # show window
    cv2.imshow("Face Centering", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# ---------------------- SHUTDOWN --------------------------
cap.release()
if out:
    out.release()
cv2.destroyAllWindows()
arduino.close()

In [3]:
# FIRST WORKING DONT CHANGE XY OFFSET CENTER Jan 02 2026 – Servo-integrated face tracking (Optimized OpenCV + Arduino gimbal control 1x Servo test)  
# ✓ smoother servo motion (PID-lite smoothing)
# ✓ speed-based gimbal slewing
# ✓ auto-recenter when target lost
# ✓ tracking memory (keeps following same face)
# ✓ recording button
# ✓ clean, safe, optimized code
# ✓ identical behavior to a targeting pod / stabilized turret

import cv2
import datetime
import os
import serial
import time

# ----------------- SERVO + SERIAL SETUP -----------------
arduino = serial.Serial('COM7', 115200)
time.sleep(0.2)

# -------- EMERGENCY PITCH RECOVERY ----------
RECOVERY_TIME = 2.0  # seconds
start = time.time()

while time.time() - start < RECOVERY_TIME:
    arduino.write(b"90,150\n")   # force center
    time.sleep(0.02)

servo_x_angle = 90
servo_y_angle = 150   # start at your desired “center” for pitch

# ---- SERVO SAFE LIMITS ----
# ---- SAFE SERVO LIMITS ----
YAW_MIN, YAW_MAX = 20, 160
PITCH_MIN, PITCH_MAX = 40, 170   # updated limits


# PID-lite smoothing factors
SMOOTH = 0.15        # 0–1, lower = smoother (less jitter)
SLEW_SPEED = 5       # max deg per frame for gimbal motion
DEADZONE = 20        # pixels to ignore jitter

# ---------------------------------------------------------


# ---------------------- RECORDING SETUP -------------------
if os.path.exists("D:\\"):
    save_dir = "D:\\Recordings"
else:
    save_dir = "C:\\recordings"
os.makedirs(save_dir, exist_ok=True)
# ----------------------------------------------------------


# ---------------------- CAMERA SETUP ----------------------
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
center_x, center_y = frame_width // 2, frame_height // 2

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
# ----------------------------------------------------------


last_face_center = None
recording = False
out = None

boot_lock = True
boot_timer = time.time()


# ---------------------- UI BUTTON -------------------------
button_w, button_h = 120, 50
button_x = (frame_width - button_w) // 2
button_y = frame_height - button_h - 20
button_color = (0, 0, 255)
# ----------------------------------------------------------


def mouse_click(event, x, y, flags, param):
    """Handles recording start/stop."""
    global recording, out, button_color

    if event == cv2.EVENT_LBUTTONDOWN:
        if button_x <= x <= button_x + button_w and button_y <= y <= button_y + button_h:
            if not recording:
                filename = f"recording_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.avi"
                filepath = os.path.join(save_dir, filename)
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                out = cv2.VideoWriter(filepath, fourcc, 20.0, (frame_width, frame_height))

                recording = True
                button_color = (0, 255, 0)
                print(f"🎥 Recording started: {filepath}")

            else:
                recording = False
                button_color = (0, 0, 255)
                if out:
                    out.release()
                    out = None
                print("🟥 Recording stopped.")


cv2.namedWindow("Face Centering")
cv2.setMouseCallback("Face Centering", mouse_click)


# ---------------------- MAIN LOOP -------------------------
while True:
    if boot_lock:
        arduino.write(b"90,150\n")
        if time.time() - boot_timer > 3:
            boot_lock = False
        continue

    ret, frame = cap.read()

    if not ret:
        break

    clean_frame = frame.copy()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=4, minSize=(60, 60)
    )

    face_center = None

    # ---------------- FACE DETECTED ----------------------
    if len(faces) > 0:

        # pick face closest to last tracked position (stable lock)
        if last_face_center is not None:
            x, y, w, h = min(
                faces,
                key=lambda f: abs((f[0] + f[2] / 2) - last_face_center[0]) +
                              abs((f[1] + f[3] / 2) - last_face_center[1])
            )
        else:
            x, y, w, h = max(faces, key=lambda rect: rect[2] * rect[3])

        # calculate face center
        face_center = (x + w // 2, y + h // 2)
        last_face_center = face_center

        # draw detection box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(frame, face_center, 5, (0, 255, 255), -1)

        # ---------------- GIMBAL LOGIC ---------------------
        error_x = face_center[0] - center_x
        error_y = face_center[1] - center_y

        # ignore tiny noise
        if abs(error_x) < DEADZONE:
            error_x = 0
        if abs(error_y) < DEADZONE:
            error_y = 0

        # invert X direction for servo correctness
        speed_x = -(error_x / center_x) * SLEW_SPEED  
        speed_y = (error_y / center_y) * SLEW_SPEED  # invert if needed

        # apply smoothing (PID-lite)
        servo_x_angle += speed_x * SMOOTH
        servo_y_angle += speed_y * SMOOTH

        # servo boundaries
        # ---- SAFE SERVO LIMITS ----
        servo_x_angle = max(YAW_MIN, min(YAW_MAX, servo_x_angle))
        servo_y_angle = max(PITCH_MIN, min(PITCH_MAX, servo_y_angle))


        # send to Arduino
        print("Servo:", int(servo_x_angle), int(servo_y_angle), "Errors:", error_x, error_y)
        arduino.write(f"{int(servo_x_angle)},{int(servo_y_angle)}\n".encode())


        # HUD
        cv2.putText(frame, f"Offset X: {error_x}", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Offset Y: {error_y}", (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # ---------------- NO FACE FOUND ----------------------
    else:
        cv2.putText(frame, "No face detected", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Slowly drift back to center
        # Slowly drift back to center
        servo_x_angle += (90 - servo_x_angle) * 0.02    # yaw still centers at 90
        servo_y_angle += (150 - servo_y_angle) * 0.02   # pitch centers at 150


        print("Servo:", int(servo_x_angle), int(servo_y_angle), "Errors: 0 0")
        arduino.write(f"{int(servo_x_angle)},{int(servo_y_angle)}\n".encode())


        last_face_center = None

    # ---------------- RECORDING BUTTON -------------------
    cv2.rectangle(frame, (button_x, button_y),
                  (button_x + button_w, button_y + button_h), button_color, -1)

    cv2.putText(frame, "Record" if not recording else "Stop",
                (button_x + 10, button_y + 35),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    if recording and out is not None:
        out.write(clean_frame)

    # show window
    cv2.imshow("Face Centering", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# ---------------------- SHUTDOWN --------------------------
cap.release()
if out:
    out.release()
cv2.destroyAllWindows()
arduino.close()

Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: -208 0
Servo: 90 150 Errors: -208 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo: 90 150 Errors: 0 0
Servo:

KeyboardInterrupt: 

In [ ]:
# FIRST WORKING DONT CHANGE X OFFSET CENTER Dec 01 2025 – Servo-integrated face tracking (Optimized OpenCV + Arduino gimbal control 1x Servo test)  
# ✓ smoother servo motion (PID-lite smoothing)
# ✓ speed-based gimbal slewing
# ✓ auto-recenter when target lost
# ✓ tracking memory (keeps following same face)
# ✓ recording button
# ✓ clean, safe, optimized code
# ✓ identical behavior to a targeting pod / stabilized turret

import cv2
import datetime
import os
import serial
import time

# ----------------- SERVO + SERIAL SETUP -----------------
arduino = serial.Serial('COM7', 115200)
time.sleep(2)

servo_x_angle = 90
servo_y_angle = 90

# PID-lite smoothing factors
SMOOTH = 0.15        # 0–1, lower = smoother (less jitter)
SLEW_SPEED = 5       # max deg per frame for gimbal motion
DEADZONE = 20        # pixels to ignore jitter

# ---------------------------------------------------------


# ---------------------- RECORDING SETUP -------------------
if os.path.exists("D:\\"):
    save_dir = "D:\\Recordings"
else:
    save_dir = "C:\\recordings"
os.makedirs(save_dir, exist_ok=True)
# ----------------------------------------------------------


# ---------------------- CAMERA SETUP ----------------------
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
center_x, center_y = frame_width // 2, frame_height // 2

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
# ----------------------------------------------------------


last_face_center = None
recording = False
out = None

# ---------------------- UI BUTTON -------------------------
button_w, button_h = 120, 50
button_x = (frame_width - button_w) // 2
button_y = frame_height - button_h - 20
button_color = (0, 0, 255)
# ----------------------------------------------------------


def mouse_click(event, x, y, flags, param):
    """Handles recording start/stop."""
    global recording, out, button_color

    if event == cv2.EVENT_LBUTTONDOWN:
        if button_x <= x <= button_x + button_w and button_y <= y <= button_y + button_h:
            if not recording:
                filename = f"recording_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.avi"
                filepath = os.path.join(save_dir, filename)
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                out = cv2.VideoWriter(filepath, fourcc, 20.0, (frame_width, frame_height))

                recording = True
                button_color = (0, 255, 0)
                print(f"🎥 Recording started: {filepath}")

            else:
                recording = False
                button_color = (0, 0, 255)
                if out:
                    out.release()
                    out = None
                print("🟥 Recording stopped.")


cv2.namedWindow("Face Centering")
cv2.setMouseCallback("Face Centering", mouse_click)


# ---------------------- MAIN LOOP -------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    clean_frame = frame.copy()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=4, minSize=(60, 60)
    )

    face_center = None

    # ---------------- FACE DETECTED ----------------------
    if len(faces) > 0:

        # pick face closest to last tracked position (stable lock)
        if last_face_center is not None:
            x, y, w, h = min(
                faces,
                key=lambda f: abs((f[0] + f[2] / 2) - last_face_center[0]) +
                              abs((f[1] + f[3] / 2) - last_face_center[1])
            )
        else:
            x, y, w, h = max(faces, key=lambda rect: rect[2] * rect[3])

        # calculate face center
        face_center = (x + w // 2, y + h // 2)
        last_face_center = face_center

        # draw detection box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(frame, face_center, 5, (0, 255, 255), -1)

        # ---------------- GIMBAL LOGIC ---------------------
        error_x = face_center[0] - center_x
        error_y = face_center[1] - center_y

        # ignore tiny noise
        if abs(error_x) < DEADZONE:
            error_x = 0
        if abs(error_y) < DEADZONE:
            error_y = 0

        # invert X direction for servo correctness
        speed_x = -(error_x / center_x) * SLEW_SPEED  
        speed_y =  (error_y / center_y) * SLEW_SPEED  # invert if needed

        # apply smoothing (PID-lite)
        servo_x_angle += speed_x * SMOOTH
        servo_y_angle += speed_y * SMOOTH

        # servo boundaries
        servo_x_angle = max(0, min(180, servo_x_angle))
        servo_y_angle = max(0, min(180, servo_y_angle))

        # send to Arduino
        print("Servo:", int(servo_x_angle), int(servo_y_angle), "Errors:", error_x, error_y)
        arduino.write(f"{int(servo_x_angle)},{int(servo_y_angle)}\n".encode())


        # HUD
        cv2.putText(frame, f"Offset X: {error_x}", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Offset Y: {error_y}", (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # ---------------- NO FACE FOUND ----------------------
    else:
        cv2.putText(frame, "No face detected", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Slowly drift back to center
        servo_x_angle += (90 - servo_x_angle) * 0.02
        servo_y_angle += (90 - servo_y_angle) * 0.02

        print("Servo:", int(servo_x_angle), int(servo_y_angle), "Errors: 0 0")
        arduino.write(f"{int(servo_x_angle)},{int(servo_y_angle)}\n".encode())


        last_face_center = None

    # ---------------- RECORDING BUTTON -------------------
    cv2.rectangle(frame, (button_x, button_y),
                  (button_x + button_w, button_y + button_h), button_color, -1)

    cv2.putText(frame, "Record" if not recording else "Stop",
                (button_x + 10, button_y + 35),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    if recording and out is not None:
        out.write(clean_frame)

    # show window
    cv2.imshow("Face Centering", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# ---------------------- SHUTDOWN --------------------------
cap.release()
if out:
    out.release()
cv2.destroyAllWindows()
arduino.close()

Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0
Servo: 90 90 Errors: 0 0


In [ ]:
# Dec 01 2025 – Servo-integrated face tracking (Optimized OpenCV + Arduino gimbal control 1x Servo test) 
# ✓ smoother servo motion (PID-lite smoothing)
# ✓ speed-based gimbal slewing
# ✓ auto-recenter when target lost
# ✓ tracking memory (keeps following same face)
# ✓ recording button
# ✓ clean, safe, optimized code
# ✓ identical behavior to a targeting pod / stabilized turret

import cv2
import datetime
import os
import serial
import time

# ----------------- SERVO + SERIAL SETUP -----------------
arduino = serial.Serial('COM7', 115200)
time.sleep(2)

servo_x_angle = 90
servo_y_angle = 90

# PID-lite smoothing factors
SMOOTH = 0.15        # 0–1, lower = smoother (less jitter)
SLEW_SPEED = 5       # max deg per frame for gimbal motion
DEADZONE = 20        # pixels to ignore jitter

# ---------------------------------------------------------


# ---------------------- RECORDING SETUP -------------------
if os.path.exists("D:\\"):
    save_dir = "D:\\Recordings"
else:
    save_dir = "C:\\recordings"
os.makedirs(save_dir, exist_ok=True)
# ----------------------------------------------------------


# ---------------------- CAMERA SETUP ----------------------
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
center_x, center_y = frame_width // 2, frame_height // 2

face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)
# ----------------------------------------------------------


last_face_center = None
recording = False
out = None

# ---------------------- UI BUTTON -------------------------
button_w, button_h = 120, 50
button_x = (frame_width - button_w) // 2
button_y = frame_height - button_h - 20
button_color = (0, 0, 255)
# ----------------------------------------------------------


def mouse_click(event, x, y, flags, param):
    """Handles recording start/stop."""
    global recording, out, button_color

    if event == cv2.EVENT_LBUTTONDOWN:
        if button_x <= x <= button_x + button_w and button_y <= y <= button_y + button_h:
            if not recording:
                filename = f"recording_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.avi"
                filepath = os.path.join(save_dir, filename)
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                out = cv2.VideoWriter(filepath, fourcc, 20.0, (frame_width, frame_height))

                recording = True
                button_color = (0, 255, 0)
                print(f"🎥 Recording started: {filepath}")

            else:
                recording = False
                button_color = (0, 0, 255)
                if out:
                    out.release()
                    out = None
                print("🟥 Recording stopped.")


cv2.namedWindow("Face Centering")
cv2.setMouseCallback("Face Centering", mouse_click)


# ---------------------- MAIN LOOP -------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    clean_frame = frame.copy()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray, scaleFactor=1.1, minNeighbors=4, minSize=(60, 60)
    )

    face_center = None

    # ---------------- FACE DETECTED ----------------------
    if len(faces) > 0:

        # pick face closest to last tracked position (stable lock)
        if last_face_center is not None:
            x, y, w, h = min(
                faces,
                key=lambda f: abs((f[0] + f[2] / 2) - last_face_center[0]) +
                              abs((f[1] + f[3] / 2) - last_face_center[1])
            )
        else:
            x, y, w, h = max(faces, key=lambda rect: rect[2] * rect[3])

        # calculate face center
        face_center = (x + w // 2, y + h // 2)
        last_face_center = face_center

        # draw detection box
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(frame, face_center, 5, (0, 255, 255), -1)

        # ---------------- GIMBAL LOGIC ---------------------

        error_x = face_center[0] - center_x
        error_y = face_center[1] - center_y

        # ignore tiny noise
        if abs(error_x) < DEADZONE:
            error_x = 0
        if abs(error_y) < DEADZONE:
            error_y = 0

        # target pod style slew: convert error → speed
        speed_x = (error_x / center_x) * SLEW_SPEED
        speed_y = (error_y / center_y) * SLEW_SPEED

        # apply smoothing (PID-lite)
        servo_x_angle += speed_x * SMOOTH
        servo_y_angle += speed_y * SMOOTH

        # servo boundaries
        servo_x_angle = max(0, min(180, servo_x_angle))
        servo_y_angle = max(0, min(180, servo_y_angle))

        # send to Arduino
        arduino.write(f"{int(servo_x_angle)},{int(servo_y_angle)}\n".encode())

        # HUD
        cv2.putText(frame, f"Offset X: {error_x}", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Offset Y: {error_y}", (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # ---------------- NO FACE FOUND ----------------------
    else:
        cv2.putText(frame, "No face detected", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Slowly drift back to center
        servo_x_angle += (90 - servo_x_angle) * 0.02
        servo_y_angle += (90 - servo_y_angle) * 0.02

        arduino.write(f"{int(servo_x_angle)},{int(servo_y_angle)}\n".encode())

        last_face_center = None

    # ---------------- RECORDING BUTTON -------------------
    cv2.rectangle(frame, (button_x, button_y),
                  (button_x + button_w, button_y + button_h), button_color, -1)

    cv2.putText(frame, "Record" if not recording else "Stop",
                (button_x + 10, button_y + 35),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    if recording and out is not None:
        out.write(clean_frame)

    # show window
    cv2.imshow("Face Centering", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# ---------------------- SHUTDOWN --------------------------
cap.release()
if out:
    out.release()
cv2.destroyAllWindows()
arduino.close()

KeyboardInterrupt: 

In [ ]:
# Dec 01 2025, Servo-integrated test face tracking (OpenCV + Arduino on COM7 + 1x Servo test)

import cv2
import datetime
import os
import serial
import time

# ----------------- SERVO SETUP -----------------
arduino = serial.Serial('COM7', 115200)  # your port
time.sleep(2)

servo_x_angle = 90  # start centered
servo_y_angle = 90
Kp = 0.05  # servo sensitivity (tune between 0.03 - 0.07)

# ------------------------------------------------

# Configuration  
if os.path.exists("D:\\"):
    save_dir = "D:\\Recordings"
else:
    save_dir = "C:\\recordings"
os.makedirs(save_dir, exist_ok=True)

# Initialize webcam
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
center_x, center_y = frame_width // 2, frame_height // 2

# Load Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Variables
last_face_center = None
recording = False
out = None

# Button setup
button_w, button_h = 120, 50
button_x = (frame_width - button_w) // 2
button_y = frame_height - button_h - 20
button_color = (0, 0, 255)  # red = not recording

def mouse_click(event, x, y, flags, param):
    global recording, out, button_color
    if event == cv2.EVENT_LBUTTONDOWN:
        if button_x <= x <= button_x + button_w and button_y <= y <= button_y + button_h:
            if not recording:
                filename = f"recording_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.avi"
                filepath = os.path.join(save_dir, filename)
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                out = cv2.VideoWriter(filepath, fourcc, 20.0, (frame_width, frame_height))
                recording = True
                button_color = (0, 255, 0)
                print(f"🎥 Recording started: {filepath}")
            else:
                recording = False
                button_color = (0, 0, 255)
                if out:
                    out.release()
                    out = None
                print("🟥 Recording stopped.")

cv2.namedWindow("Face Centering")
cv2.setMouseCallback("Face Centering", mouse_click)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    clean_frame = frame.copy()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4, minSize=(60, 60))

    face_center = None
    x = y = w = h = None

    if len(faces) > 0:
        # Pick closest face to previous or largest face
        if last_face_center is not None:
            x, y, w, h = min(
                faces,
                key=lambda f: abs((f[0] + f[2] / 2) - last_face_center[0]) +
                              abs((f[1] + f[3] / 2) - last_face_center[1])
            )
        else:
            x, y, w, h = max(faces, key=lambda rect: rect[2] * rect[3])

        # Compute face center
        face_center = (x + w // 2, y + h // 2)
        last_face_center = face_center

        # Draw box + dot
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(frame, face_center, 5, (0, 255, 255), -1)

        # ----- TARGETING POD STYLE SERVO CONTROL -----

        deadzone = 20  # ignore tiny jitter

        # Error in pixels
        error_x = face_center[0] - center_x
        error_y = face_center[1] - center_y

        # Servo movement speed based on how far the face is
        speed_x = int((error_x / center_x) * 5) if abs(error_x) > deadzone else 0
        speed_y = int((error_y / center_y) * 5) if abs(error_y) > deadzone else 0

        # Apply movement
        servo_x_angle += speed_x
        servo_y_angle += speed_y

        # Limit
        servo_x_angle = max(0, min(180, servo_x_angle))
        servo_y_angle = max(0, min(180, servo_y_angle))

        # Send to Arduino
        arduino.write(f"{servo_x_angle},{servo_y_angle}\n".encode())

        # HUD
        direction_x = "Left" if error_x < 0 else "Right"
        direction_y = "Up" if error_y < 0 else "Down"


        cv2.putText(frame, f"Offset X: {error_x} ({direction_x})", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Offset Y: {error_y} ({direction_y})", (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    else:
        cv2.putText(frame, "No face detected", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.rectangle(frame, (button_x, button_y), (button_x + button_w, button_y + button_h), button_color, -1)
    cv2.putText(frame, "Record" if not recording else "Stop",
                (button_x + 10, button_y + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    if recording and out is not None:
        out.write(clean_frame)

    cv2.imshow("Face Centering", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
if out:
    out.release()
cv2.destroyAllWindows()
arduino.close()


KeyboardInterrupt: 

In [1]:
# Nov 01 2025, (REVISED TRACKING, WITH RECORDING FUNCTION, WIP) FACE TRACKING WEBCAM SCRIPT (OpenCV + HAARCASCADES)
# Keeps face centered on screen & calculates offset
import cv2
import datetime
import os

# Configuration  
if os.path.exists("D:\\"):
    save_dir = "D:\\Recordings"
else:
    save_dir = "C:\\recordings"
os.makedirs(save_dir, exist_ok=True)

# Initialize webcam
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
center_x, center_y = frame_width // 2, frame_height // 2

# Load Haar Cascade (optional: replace with DNN)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Variables
last_face_center = None
recording = False
out = None

# Button setup
button_w, button_h = 120, 50
button_x = (frame_width - button_w) // 2
button_y = frame_height - button_h - 20
button_color = (0, 0, 255)  # red = not recording
# button_x, button_y, button_w, button_h = 20, 20, 120, 50
# button_color = (0, 0, 255)  # red = not recording

def mouse_click(event, x, y, flags, param):
    """Handle mouse clicks on the record button."""
    global recording, out, button_color
    if event == cv2.EVENT_LBUTTONDOWN:
        if button_x <= x <= button_x + button_w and button_y <= y <= button_y + button_h:
            if not recording:
                # Start recording
                filename = f"recording_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.avi"
                filepath = os.path.join(save_dir, filename)
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                out = cv2.VideoWriter(filepath, fourcc, 20.0, (frame_width, frame_height))
                recording = True
                button_color = (0, 255, 0)
                print(f"🎥 Recording started: {filepath}")
            else:
                # Stop recording
                recording = False
                button_color = (0, 0, 255)
                if out:
                    out.release()
                    out = None
                print("🟥 Recording stopped.")

cv2.namedWindow("Face Centering")
cv2.setMouseCallback("Face Centering", mouse_click)

while True:
    ret, frame = cap.read() # Single frame read
    if not ret: # Dip from loop if fail
        break

    # Make a clean copy for recording (no overlays)
    clean_frame = frame.copy()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # RGB to Grayscale for face detection
    
    # PENTING: FACE DETECTION PARAMETERS CHANGE IF SEE FIT
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4, minSize=(60, 60))

    # Default no-face centering params
    face_center = None 
    x = y = w = h = None

    if len(faces) > 0:
        if last_face_center is not None:
            x, y, w, h = min(
                faces,
                key=lambda f: abs((f[0] + f[2] / 2) - last_face_center[0]) +
                              abs((f[1] + f[3] / 2) - last_face_center[1])
            )
        else:
            x, y, w, h = max(faces, key=lambda rect: rect[2] * rect[3])

        last_face_center = (x + w // 2, y + h // 2)
        face_center = (x + w // 2, y + h // 2)

        # Draw tracking box and center dot (only for preview, not in recording)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(frame, face_center, 5, (0, 255, 255), -1)

    if face_center is not None:
        error_x = face_center[0] - center_x
        error_y = face_center[1] - center_y
        direction_x = "Left" if error_x < 0 else "Right"
        direction_y = "Up" if error_y < 0 else "Down"

        # SERVO CODE HERE:

        # HUD text overlay.
        cv2.putText(frame, f"Offset X: {error_x} ({direction_x})", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Offset Y: {error_y} ({direction_y})", (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    else:
        cv2.putText(frame, "No face detected", (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    #   Draw the interactive record button (on preview only)  
    cv2.rectangle(frame, (button_x, button_y), (button_x + button_w, button_y + button_h), button_color, -1)
    cv2.putText(frame, "Record" if not recording else "Stop",
                (button_x + 10, button_y + 35), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    #   Save the clean frame if recording
    if recording and out is not None:
        out.write(clean_frame)

    # Show webcam preview
    cv2.imshow("Face Centering", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
if out:
    out.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 

In [7]:
# servo control code after the following block

if face_center is not None:
    error_x = face_center[0] - center_x
    error_y = face_center[1] - center_y
    direction_x = "Left" if error_x < 0 else "Right"
    direction_y = "Up" if error_y < 0 else "Down"


NameError: name 'face_center' is not defined

In [ ]:
# Oct 31 2025, (RECORDING FUNCTION, SCRAPPED) FACE TRACKING WEBCAM SCRIPT (OpenCV + HAARCASCADES)
# Keeps face centered on screen & calculates offset

import cv2
import numpy as np
import datetime

# Setup webcam  
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
center_x, center_y = frame_width // 2, frame_height // 2

# Load face detector  
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Tracking memory  
last_face_center = None

# Recording setup  
is_recording = False          # Flag: currently recording?
out = None                    # VideoWriter (will be created when recording starts)

print("Press 'r' to start/stop recording, 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4, minSize=(60, 60))

    face_center = None
    x = y = w = h = 0

    if len(faces) > 0:
        # Sticky tracking: choose face closest to previous frame
        if last_face_center is not None:
            x, y, w, h = min(
                faces,
                key=lambda f: abs((f[0] + f[2] / 2) - last_face_center[0]) +
                              abs((f[1] + f[3] / 2) - last_face_center[1])
            )
        else:
            # First detection → largest face
            x, y, w, h = max(faces, key=lambda rect: rect[2] * rect[3])

        last_face_center = (x + w // 2, y + h // 2)
        face_center = (x + w // 2, y + h // 2)

        # Draw bounding box + center dot
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(frame, face_center, 5, (0, 255, 255), -1)

    # Compute tracking offset & direction  
    if face_center is not None:
        error_x = face_center[0] - center_x
        error_y = face_center[1] - center_y
        direction_x = "Left" if error_x < 0 else "Right"
        direction_y = "Up" if error_y < 0 else "Down"

        cv2.putText(frame, f"Offset X: {error_x} ({direction_x})", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Offset Y: {error_y} ({direction_y})", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    else:
        cv2.putText(frame, "No face detected", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Recording overlay  
    if is_recording:
        cv2.putText(frame, "REC", (frame_width - 80, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
        out.write(frame)  # save frame to file

    # Display the frame  
    cv2.imshow("Face Tracker", frame)

    # Handle keyboard input  
    key = cv2.waitKey(1) & 0xFF

    if key == ord('r'):
        # Toggle recording on/off
        if not is_recording:
            # Start a new recording file with timestamp
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            filename = f"recording_{timestamp}.avi"
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            out = cv2.VideoWriter(filename, fourcc, 20.0, (frame_width, frame_height))
            is_recording = True
            print(f"🎥 Started recording: {filename}")
        else:
            # Stop recording
            is_recording = False
            out.release()
            out = None
            print("🛑 Stopped recording")

    elif key == ord('q'):
        break

# Cleanup  
cap.release()
if out is not None:
    out.release()
cv2.destroyAllWindows()


Press 'r' to start/stop recording, 'q' to quit.


KeyboardInterrupt: 

In [ ]:
# Oct 31 2025, (RAW REVISED TRACKING, WITHOUT RECORDING FUNCTION, WIP) FACE TRACKING WEBCAM SCRIPT (OpenCV + HAARCASCADES)
# Keeps face centered on screen & calculates offset

import cv2  # OpenCV library

 
# 1. Initialize the Webcam
 
# 0 = default camera
cap = cv2.VideoCapture(0)

# Check if webcam opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

 
# 2. Load Face Detection Model
 
# Haar cascades are pretrained XML files for object detection
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

 
# 3. Frame Size (reference center)
 
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# center of the frame
center_x = frame_width // 2
center_y = frame_height // 2

print(f"Frame Center: ({center_x}, {center_y})")

# ASSUME: cap, face detector, frame_width, frame_height, center_x, center_y are defined above  
# Initialize the sticky-tracking variable (before the loop)
last_face_center = None

while True:
    ret, frame = cap.read() # Single frame read
    if not ret: # Dip from loop if fail
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # RGB to Grayscale for face detection

    # PENTING: FACE DETECTION PARAMETERS CHANGE IF SEE FIT
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4, minSize=(60, 60))

    # Default: no face found this frame
    face_center = None
    x = y = w = h = None

    if len(faces) > 0:
        # Choose the face closest to the last known one (sticky tracking)
        if last_face_center is not None:
            # Compute distance from previous center for each detected face and pick the closest
            x, y, w, h = min(
                faces,
                key=lambda f: abs((f[0] + f[2] / 2) - last_face_center[0]) +
                              abs((f[1] + f[3] / 2) - last_face_center[1])
            )
        else:
            # On the first hit, pick the largest face (most reliable)
            x, y, w, h = max(faces, key=lambda rect: rect[2] * rect[3])

        # Update the last known face center (used next frame)
        last_face_center = (x + w // 2, y + h // 2)

        # Now compute the face center for use in the rest of the loop
        face_center = (x + w // 2, y + h // 2)

        # Draw rectangle and center dot
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(frame, face_center, 5, (0, 255, 255), -1)

    # Now handle either case: found or not found
    if face_center is not None:
        # Calculate offset/error from frame center
        error_x = face_center[0] - center_x
        error_y = face_center[1] - center_y

        direction_x = "Left" if error_x < 0 else "Right"
        direction_y = "Up" if error_y < 0 else "Down"

        cv2.putText(frame, f"Offset X: {error_x} ({direction_x})", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        cv2.putText(frame, f"Offset Y: {error_y} ({direction_y})", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        # Replace this with serial/servo mapping later
        print(f"Move {direction_x} {abs(error_x)} | Move {direction_y} {abs(error_y)}")
    else:
        # No face detected this frame — don't crash or reference undefined variables
        cv2.putText(frame, "No face detected", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Show frame and handle quit key
    cv2.imshow("Face Centering", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# cleanup after loop
cap.release()
cv2.destroyAllWindows()

Frame Center: (320, 240)
Move Right 54 | Move Down 86
Move Right 54 | Move Down 86
Move Right 54 | Move Down 86
Move Right 54 | Move Down 86
Move Right 56 | Move Down 87
Move Right 56 | Move Down 87
Move Right 55 | Move Down 84
Move Right 55 | Move Down 84
Move Right 56 | Move Down 85
Move Right 56 | Move Down 85
Move Right 64 | Move Down 84
Move Right 64 | Move Down 84
Move Right 63 | Move Down 85
Move Right 63 | Move Down 85
Move Right 71 | Move Down 41
Move Right 71 | Move Down 41
Move Right 68 | Move Down 78
Move Right 68 | Move Down 78
Move Right 72 | Move Down 81
Move Right 72 | Move Down 81
Move Right 70 | Move Down 77
Move Right 70 | Move Down 77
Move Right 67 | Move Down 75
Move Right 67 | Move Down 75
Move Right 62 | Move Down 73
Move Right 62 | Move Down 73
Move Right 54 | Move Down 68
Move Right 54 | Move Down 68
Move Right 50 | Move Down 71
Move Right 50 | Move Down 71
Move Right 50 | Move Down 69
Move Right 46 | Move Down 70
Move Right 43 | Move Down 70
Move Right 48 | Mo

KeyboardInterrupt: 

In [ ]:
# (PRIMITIVE) FACE TRACKING WEBCAM SCRIPT (OpenCV)
# Keeps face centered on screen & calculates offset

import cv2  # OpenCV library

 
# 1. Initialize the Webcam
 
# 0 = default camera
cap = cv2.VideoCapture(0)

# Check if webcam opened successfully
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

 
# 2. Load Face Detection Model
 
# Haar cascades are pretrained XML files for object detection
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

 
# 3. Frame Size (reference center)
 
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# center of the frame
center_x = frame_width // 2
center_y = frame_height // 2

print(f"Frame Center: ({center_x}, {center_y})")

# Initialize last known face center (for tracking consistency)
last_face_center = None

 
# 4. Main Loop — Process Each Frame
 
while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break  # Exit loop if no frame is captured

    # Convert frame to grayscale (faster for detection)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(
    gray,
    scaleFactor=1.1,  # Lower = more sensitive (1.1 detects smaller changes)
    minNeighbors=3,   # Lower = detects more but less accurate
    minSize=(60, 60)  # Ignore tiny detections (noise)
)

    # Draw a blue dot at the center of the frame
    cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)

     
    # 5. If a Face is Detected
     
    if len(faces) > 0:
        # Select the largest face (useful if multiple are detected)
        (x, y, w, h) = max(faces, key=lambda rect: rect[2] * rect[3])

        # Find the center of that face
        face_center = (x + w // 2, y + h // 2)

        # Draw a green rectangle around the detected face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Draw a yellow dot at the face center
        cv2.circle(frame, face_center, 5, (0, 255, 255), -1)

         
        # 6. Calculate Face Offset (Error)
         
        error_x = face_center[0] - center_x
        error_y = face_center[1] - center_y

        # Determine direction based on offset
        direction_x = "Left" if error_x < 0 else "Right"
        direction_y = "Up" if error_y < 0 else "Down"

        # Display the offset and direction on screen
        cv2.putText(
            frame,
            f"Offset X: {error_x} ({direction_x})",
            (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 255),
            2,
        )
        cv2.putText(
            frame,
            f"Offset Y: {error_y} ({direction_y})",
            (10, 60),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 255),
            2,
        )

        # Print movement command (robotic coords)
        print(f"Move {direction_x} {abs(error_x)} | Move {direction_y} {abs(error_y)}")

    else:
        # 7. If No Face is Found
        cv2.putText(
            frame,
            "No face detected",
            (10, 30),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 0, 255),
            2,
        )

    # 8. Display Result
    cv2.imshow("Face Centering", frame)

    # Press 'q' to quit the live feed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 9. Cleanup — Release Resources
cap.release()
cv2.destroyAllWindows()


Frame Center: (320, 240)
Move Right 10 | Move Down 24
Move Right 8 | Move Down 18
Move Right 9 | Move Down 17
Move Right 7 | Move Down 19
Move Right 9 | Move Down 19
Move Right 5 | Move Down 20
Move Right 7 | Move Down 21
Move Right 10 | Move Down 20
Move Right 8 | Move Down 21
Move Right 12 | Move Down 20
Move Right 13 | Move Down 20
Move Right 13 | Move Down 20
Move Right 12 | Move Down 20
Move Right 11 | Move Down 22
Move Right 12 | Move Down 22
Move Right 12 | Move Down 21
Move Right 11 | Move Down 18
Move Right 11 | Move Down 23
Move Right 11 | Move Down 23
Move Right 8 | Move Down 23
Move Right 6 | Move Down 23
Move Right 3 | Move Down 23
Move Right 6 | Move Down 23
Move Right 5 | Move Down 23
Move Right 9 | Move Down 22
Move Right 8 | Move Down 22
Move Right 4 | Move Down 22
Move Right 6 | Move Down 24
Move Right 7 | Move Down 23
Move Right 7 | Move Down 23
Move Right 8 | Move Down 22
Move Right 7 | Move Down 22
Move Right 6 | Move Down 22
Move Right 8 | Move Down 22
Move Right 

KeyboardInterrupt: 